# Pipeline Tasks

<br>Owner: **Alex Drlica-Wagner** ([@kadrlica](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@kadrlica))
<br>Last Verified to Run: **2018-08-10**
<br>Verified Stack Release: **v16.0**

## Learning Objectives:

This notebook seeks to teach users how to unpack a pipeline tasks. As an example, we focus on `processCcd.py`, with the goal of diving into the configuration, interface, and structure of pipeline tasks. This notebook is a digression from Justin Myles script that demonstrates how to run a series of pipeline tasks from the command line to rerun HSC data processing [link].

After working through this tutorial you should be able to:

* Find the source code for a pipeline task
* Configure (and investigate the configuration) of pipeline tasks
* Investigate and run those tasks in python

## Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

In [ ]:
import os
import pydoc

## Diving into a Pipeline Task

Our goal is to dive into the inner workings of `ProcessCcd.py`. We pickup from the command line processing described in the getting started tutorials [here](https://pipelines.lsst.io/getting-started/data-setup.html#) and [here](https://pipelines.lsst.io/getting-started/processccd.html), as well as Justin Myles HSC reprocessing notebook [here](). We are specifically interested in digging into the following line:

```
processCcd.py $DATADIR --rerun processCcdOutputs --id
```

We start by tracking down the location of the `processCcd.py` shell script

In [ ]:
!(which processCcd.py)

This is the proverbial "end of the thread". Our goal is to pull on this thread to unravel the python/C++ functions that are being called under the hood. We start by taking a peak in this script

In [ ]:
!cat $(which processCcd.py)

Ok, this hasn't gotten us very far, but after getting through the stock header, we now have the next link in our chain:
```
from lsst.pipe.tasks.processCcd import ProcessCcdTask
```

There are two ways we can proceed from here. One is to [Google](http://lmgtfy.com/?q=lsst.pipe.tasks.processCcd) `lsst.pipe.tasks.processCcd`, which will take us to this [doxygen page](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1tasks_1_1process_ccd_1_1_process_ccd_task.html) and/or the soure code on [GitHub](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py). 

The second approach is to do the import the class oursleves and try to investigate it interactively.


In [ ]:
import lsst.pipe.tasks.processCcd
from lsst.pipe.tasks.processCcd import ProcessCcdTask, ProcessCcdConfig

We can get to the source code for these classes directly using the [`stackclub` toolkit module](https://stackclub.readthedocs.io/), as shown in the [FindingDocs.ipynb](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/GettingStarted/FindingDocs.ipynb)

In [ ]:
from stackclub import where_is
where_is(ProcessCcdConfig)

# Diving into a Task Config

Pipeline tasks are controlled and tweaked through there associated `TaskConfig` objects. To investigate the configuration parameters of the `ProcessCcdTask`, we create an instance of the `ProcessCcdConfig` and try calling the `help` method (commented out for brevity). What we are really interested in are the "Data descriptors", which we can print directly after capturing the documentation output by `help`.

In [ ]:
config = ProcessCcdConfig()
#help(config)
helplist = pydoc.render_doc(config).split('\n')
print('\n'.join(helplist[18:47]))

The first step is to try to get at the documentation through the `help` function (commented out below for brevity). We can find 

The `ProcessCcdConfig` object contains of both raw configurables like `doCalibrate` and other configs, like `isr`. To investigate one of these in more detail, we can import it and query it's "Data descriptors".


In [ ]:
from lsst.ip.isr.isrTask import IsrTask, IsrTaskConfig
print('\n'.join(pydoc.render_doc(IsrTaskConfig).split('\n')[16:40]) + '...')

These configurationas are pretty self-explanitory, but say that we really want to understand what `doFringe` is doing. Inorder to get that information we need to go to the source code.

In [ ]:
where_is(IsrTaskConfig)

We can then search this file for `doFringe` and we find [several lines](https://github.com/lsst/ip_isr/blob/cc4efb7d763d3663c9e989339505df9654f23fd9/python/lsst/ip/isr/isrTask.py#L597-L598) that look like this:

        if self.config.doFringe and not self.config.fringeAfterFlat:
            self.fringe.run(ccdExposure, **fringes.getDict())
            
If we want to go deeper to see what `fringe.run` does, we can repeat the above process

In [ ]:
isr_task = IsrTask()
print(isr_task.fringe)
import  lsst.ip.isr.fringe
where_is(lsst.ip.isr.fringe.FringeTask)

We finally make our way to the source code for [FringeTask.run](https://github.com/lsst/ip_isr/blob/cc4efb7d763d3663c9e989339505df9654f23fd9/python/lsst/ip/isr/fringe.py#L104), which gives us details on how the fringe correction is performed (i.e. by creating a fringe image and subtracting it from the data image).